In [1]:
# run the script initnotebook.py in the cuurent folder
# Error when run multiple times becasue the directory changed
%run initnotebook.py

current working dir/home/abjawad/Documents/GitHub/local-attention-model


In [2]:
import torch
from  torch.utils.data import DataLoader
import torch.nn as nn
from models.builder import EncoderDecoder as segmodel
from dataloader.cfg_defaults import get_cfg_defaults
from config_cityscapes import *
import os
from dataloader.cityscapes_dataloader import CityscapesDataset
from val_segformer_rgbonly import val_cityscape

import torch.nn.functional as F
from utils.visualize import unnormalize_img_numpy
import matplotlib.pyplot as plt

from visualizer.visualizer import *

In [3]:
config_path = 'dataloader/cityscapes_rgbd_config.yaml'
config_path = os.path.join(projectFolder, config_path)

cfg = get_cfg_defaults()
cfg.merge_from_file(config_path)
cfg.freeze()

data_mean = [0.291,  0.329,  0.291]
data_std = [0.190,  0.190,  0.185]

In [ ]:
print(cfg)

In [4]:
# train dataloader 
cityscapes_train = CityscapesDataset(cfg, split='train')
train_loader = DataLoader(cityscapes_train, batch_size=1, shuffle=False, num_workers=4,drop_last=True)
print(f'total train sample: {len(cityscapes_train)} v_iteration:{len(train_loader)}')

Using RGB input
Using RGB input
Found 2975 train images
total train sample: 2975 v_iteration:2975


In [5]:
criterion = nn.CrossEntropyLoss(reduction='mean', ignore_index=config.background)

model = segmodel(cfg=config, criterion=criterion, norm_layer=nn.BatchNorm2d)
model = nn.DataParallel(model, device_ids = config.device_ids)
model.to(f'cuda:{model.device_ids[0]}', non_blocking=True)

06 13:09:06 Using backbone: Segformer-B2
06 13:09:06 Using MLP Decoder
06 13:09:06 Loading pretrained model: /home/abjawad/Documents/GitHub/local-attention-model/pretrained/mit_b2.pth
06 13:09:06 Load model, Time usage:
	IO: 0.04514932632446289, initialize parameters: 0.017549991607666016
06 13:09:06 Initing weights ...


DataParallel(
  (module): EncoderDecoder(
    (backbone): mit_b2(
      (patch_embed1): OverlapPatchEmbed(
        (proj): Conv2d(3, 64, kernel_size=(7, 7), stride=(4, 4), padding=(3, 3))
        (norm): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
      )
      (patch_embed2): OverlapPatchEmbed(
        (proj): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        (norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      )
      (patch_embed3): OverlapPatchEmbed(
        (proj): Conv2d(128, 320, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        (norm): LayerNorm((320,), eps=1e-05, elementwise_affine=True)
      )
      (patch_embed4): OverlapPatchEmbed(
        (proj): Conv2d(320, 512, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        (norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      )
      (block1): ModuleList(
        (0): Block(
          (norm1): LayerNorm((64,), eps=1e-06, elementwise_affine=True)
       

In [ ]:
# pretrained_model_path = './pretrained/model_400.pth'
# saved_model_path = os.path.join(projectFolder, pretrained_model_path)
# print(saved_model_path)
# # exit()
# state_dict = torch.load(saved_model_path)
# model.load_state_dict(state_dict['model'], strict=False)
# print(f'model loaded')
# epoch = state_dict['epoch']


In [6]:
# create a random 8X3X1024x1024

input = torch.rand(1,3,512,512).cuda()
gt = torch.randint(0, 19, (1,1024,1024)).cuda()

In [7]:
from fvcore.nn import FlopCountAnalysis

In [8]:
flops = FlopCountAnalysis(model, input)

In [10]:
print(flops)

In [11]:
print(f'flops: {flops.total()/1e9} G')

input:  torch.Size([1, 3, 512, 512])
tokenization:  torch.Size([1, 16384, 64])


/home/abjawad/Documents/GitHub/local-attention-model/models/encoders/multi_scale_attention.py:89: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  q = self.q(x).reshape(B, N, self.num_heads, C // self.num_heads).permute(0, 2, 1, 3)
/home/abjawad/Documents/GitHub/local-attention-model/models/encoders/multi_scale_attention.py:90: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch

ValueError: not enough values to unpack (expected 4, got 2)